# Шаг 4 - Регрессионное моделирование
Построить регрессионную модель температуры воздуха в помещении в зависимости от типа охлаждения и других значимых факторов. Выбор факторов - на усмотрение исполнителя.

## Импорт датасета и либ

In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.preprocessing import OneHotEncoder, LabelBinarizer
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error, r2_score, mean_absolute_error, mean_squared_error

from IPython.display import display


df = pd.read_csv('data_3.csv', delimiter=';')
print(list(df.columns))
df.head(100)

['год', 'время_года', 'климат', 'город', 'страна', 'способ_охлаждения', 'режим_при_смешанном_типе_охлаждения', 'способ_обогрева', 'возраст', 'пол', 'ощущение_температуры', 'ощущение_температуры_(bool)', 'предпочтительное_изменение_температуры', 'ощущение_движения_воздуха_(bool)', 'предпочтительное_изменение_движения_воздуха', 'оценка_комфорта', 'утепление', 'температура_воздуха_в_помещении', 'температура_воздуха_на_улице', 'rh', 'скорость_воздуха', 'рост', 'вес', 'занавески', 'вентилятор', 'окно', 'двери', 'отопление', 'среднемесячная_температура_на_улице', 'количество_рекламаций', 'температура_удовлетворительная_%', 'количество_рекламаций_кат', 'возраст_кат', 'rh_кат']


,год,время_года,климат,город,страна,способ_охлаждения,режим_при_смешанном_типе_охлаждения,способ_обогрева,возраст,пол,...,вентилятор,окно,двери,отопление,среднемесячная_температура_на_улице,количество_рекламаций,температура_удовлетворительная_%,количество_рекламаций_кат,возраст_кат,rh_кат
0,2010,Лето,Cубтропический океанический,Сидней,Австралия,Смешанный,Вентиляция,Механическое отопление,55.0,Женский,...,0,-1,-1,0,22.0,0,56.2,мало,средний возраст,норма
1,2010,Лето,Cубтропический океанический,Сидней,Австралия,Смешанный,Вентиляция,Механическое отопление,55.0,Женский,...,0,0,0,0,22.0,0,56.2,мало,средний возраст,норма
2,2010,Лето,Cубтропический океанический,Сидней,Австралия,Смешанный,Вентиляция,Механическое отопление,45.0,Женский,...,0,-1,-1,0,22.0,0,56.2,мало,средний возраст,норма
3,2010,Лето,Cубтропический океанический,Сидней,Австралия,Смешанный,Вентиляция,Механическое отопление,45.0,Женский,...,0,0,0,0,22.0,0,56.2,мало,средний возраст,норма
4,2010,Лето,Cубтропический океанический,Сидней,Австралия,Смешанный,Вентиляция,Механическое отопление,55.0,Женский,...,0,-1,-1,0,22.0,0,56.2,мало,средний возраст,более_60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2010,Лето,Cубтропический океанический,Сидней,Австралия,Смешанный,Кондиционирование,Механическое отопление,45.0,Женский,...,0,-1,-1,0,22.0,0,56.2,мало,средний возраст,норма
96,2010,Лето,Cубтропический океанический,Сидней,Австралия,Смешанный,Кондиционирование,Механическое отопление,35.0,Женский,...,0,-1,-1,0,22.0,0,56.2,мало,молодой возраст,более_60
97,2010,Лето,Cубтропический океанический,Сидней,Австралия,Смешанный,Кондиционирование,Механическое отопление,25.0,Женский,...,0,1,-1,0,22.0,0,56.2,мало,молодой возраст,более_60
98,2010,Лето,Cубтропический океанический,Сидней,Австралия,Смешанный,Кондиционирование,Механическое отопление,55.0,Женский,...,0,0,0,0,22.0,0,56.2,мало,средний возраст,менее_40


## Моделирование

### Выбрать нужные колонки

In [56]:
cols_df_original = list(df.columns)

# Output => 'температура_воздуха_в_помещении'
col_pred = ['температура_воздуха_в_помещении']

# Using (OHE categorial)
cols_input_ohe = []
cols_input_cat = [
    'год', 'время_года', 'климат', 'город', 
    
    'способ_охлаждения', 'режим_при_смешанном_типе_охлаждения', 'способ_обогрева',
    'ощущение_температуры', 'предпочтительное_изменение_температуры', 'предпочтительное_изменение_движения_воздуха', 
    
    'rh_кат'
]

# Using
cols_input_val = [ 
    'ощущение_температуры_(bool)', 'ощущение_движения_воздуха_(bool)',
    'утепление', 'скорость_воздуха', 'температура_воздуха_на_улице', 'среднемесячная_температура_на_улице', 'температура_удовлетворительная_%', 
    
    'рост', 'вес', 'занавески', 'вентилятор', 'окно', 'двери', 'отопление', 'rh', 
]

# Not using
_ = [
    # Не влияет на температуру
    'пол',
    
    'возраст',
    'возраст_кат', 
    
    # Не имеет смысла
    'количество_рекламаций', 
    'количество_рекламаций_кат', 
    
    # Ничего не меняет
    'страна',  # Есть город
    
    # Nan
    'оценка_комфорта',
]

### Подготовка датасета

In [57]:
filter1 = ~df['режим_при_смешанном_типе_охлаждения'].isna()
filter2 = df['способ_охлаждения'].str.contains('Смешанный')
idx = df[ filter1 & filter2 ].index
df.loc[idx, 'способ_охлаждения'] = df.loc[idx, 'режим_при_смешанном_типе_охлаждения']

df['способ_обогрева'] = df['способ_обогрева'].fillna('NaN')

print(df['способ_охлаждения'].unique(), df['способ_обогрева'].unique())

['Вентиляция' 'Кондиционирование'] ['Механическое отопление' 'NaN']


In [71]:
# data = pd.get_dummies(df, prefix=[label], columns=[label], drop_first=True)
# ohe = OneHotEncoder(sparse_output=False, drop='first')
ohe = LabelBinarizer()
data = None

for col_cat in cols_input_cat:
    # print(col_cat)
    cols_input_ohe.append(col_cat)  # ???

    # ohe.fit(df[label]) transformed = ohe.transform(df[label])
    transformed = ohe.fit_transform(df[col_cat].astype(str))
    ohe_df = pd.DataFrame(transformed).add_suffix(f"_{col_cat}")
    if data is not None:
        data = pd.concat([data, ohe_df], axis=1)
    else:
        data = ohe_df


cols_input_ohe = list(data.columns)
X_cols, y_col = cols_input_val + cols_input_ohe, col_pred

data = pd.concat([df[col_pred], df[cols_input_val], data], axis=1)

data

,температура_воздуха_в_помещении,ощущение_температуры_(bool),ощущение_движения_воздуха_(bool),утепление,скорость_воздуха,температура_воздуха_на_улице,среднемесячная_температура_на_улице,температура_удовлетворительная_%,рост,вес,...,21_ощущение_температуры,0_предпочтительное_изменение_температуры,1_предпочтительное_изменение_температуры,2_предпочтительное_изменение_температуры,0_предпочтительное_изменение_движения_воздуха,1_предпочтительное_изменение_движения_воздуха,2_предпочтительное_изменение_движения_воздуха,0_rh_кат,1_rh_кат,2_rh_кат
0,21.9,1,1,0.46,0.05,24.1,22.0,56.2,165.1,65.0,...,0,1,0,0,1,0,0,0,0,1
1,22.3,1,1,0.43,0.08,24.1,22.0,56.2,165.1,65.0,...,0,1,0,0,1,0,0,0,0,1
2,23.6,-1,1,0.39,0.08,24.1,22.0,56.2,165.1,65.0,...,0,0,1,0,1,0,0,0,0,1
3,24.3,1,-1,0.52,0.09,24.1,22.0,56.2,165.1,65.0,...,0,1,0,0,0,1,0,0,0,1
4,22.5,-1,-1,0.71,0.10,24.1,22.0,56.2,165.1,65.0,...,0,0,1,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,NaN,1,0,0.61,0.12,24.1,28.9,50.0,165.1,65.0,...,0,1,0,0,1,0,0,0,1,0
478,24.3,1,0,0.61,0.11,24.1,28.9,50.0,165.1,65.0,...,0,1,0,0,1,0,0,0,1,0
479,24.1,1,0,0.61,0.14,23.9,28.9,50.0,165.1,65.0,...,0,1,0,0,1,0,0,0,1,0
480,24.1,1,0,0.61,0.12,23.9,28.9,50.0,165.1,65.0,...,0,1,0,0,1,0,0,0,1,0


In [75]:
# Drop: температура_воздуха_в_помещении (10 nan) не помогут при обучении модели
data_all = data[data.columns]
display(data_all[data_all.isnull().any(axis=1)].head(1))

data_all.dropna(axis=0, how='any', inplace=True)
# data[col_pred] = data[col_pred].fillna(data[col_pred].median())

,температура_воздуха_в_помещении,ощущение_температуры_(bool),ощущение_движения_воздуха_(bool),утепление,скорость_воздуха,температура_воздуха_на_улице,среднемесячная_температура_на_улице,температура_удовлетворительная_%,рост,вес,...,21_ощущение_температуры,0_предпочтительное_изменение_температуры,1_предпочтительное_изменение_температуры,2_предпочтительное_изменение_температуры,0_предпочтительное_изменение_движения_воздуха,1_предпочтительное_изменение_движения_воздуха,2_предпочтительное_изменение_движения_воздуха,0_rh_кат,1_rh_кат,2_rh_кат
413,NaN,-1,0,0.57,0.07,24.7,32.8,50.0,165.1,65.0,...,0,0,0,1,0,1,0,0,1,0


### Вычисления и метрики точности

In [76]:
data_train, data_test = train_test_split(data_all, test_size=0.2, random_state=123)

linear_reg = LinearRegression()
linear_reg.fit(data_train[X_cols], data_train[y_col])

y_pred = linear_reg.predict(data_test[X_cols])
y_true = data_test[y_col]

# y_true = pd.DataFrame(y_true).reset_index(drop=True)
y_true = np.array(y_true)

print(f"""
R2  : {r2_score(y_true, y_pred):.2f}
MSE : {mean_squared_error(y_true, y_pred):.2f}
MAE : {mean_absolute_error(y_true, y_pred):.2f} C
MAPE: {mean_absolute_percentage_error(y_true, y_pred) * 100:.2f} %
""".strip())

# print(y_pred, y_true)
# print(pd.concat([pd.DataFrame(y_pred), y_true]))
# print(np.concatenate([y_pred, y_true], axis=1))
# print(sorted(data_train.index), sorted(data_test.index), sep='\n')

R2  : 0.71
MSE : 2.66
MAE : 1.23 C
MAPE: 5.00 %


In [77]:
for val, pred in zip(y_true, y_pred):
    val = val[0]
    pred = round(float(pred[0]), 3)
    print(val, pred)

24.0 24.582
24.9 23.755
24.8 24.46
24.4 24.298
21.4 21.362
25.7 22.614
25.0 25.279
24.6 27.4
29.6 29.228
24.3 24.097
20.5 22.206
24.7 24.175
28.2 25.265
23.8 24.559
24.3 24.895
24.6 25.313
24.3 24.24
23.7 24.733
22.5 22.961
37.4 33.431
26.5 26.213
26.1 27.861
24.3 23.851
21.1 22.978
25.1 25.316
24.4 25.995
25.7 25.721
27.7 26.641
23.8 24.959
25.5 24.202
29.3 26.528
25.3 23.501
24.6 24.201
22.4 23.778
24.4 24.377
21.3 22.916
24.3 26.213
26.9 25.229
30.4 27.737
25.7 25.957
22.0 21.208
28.0 28.899
25.2 24.125
24.6 24.358
24.0 23.943
24.2 24.005
23.3 25.95
26.1 26.931
23.6 23.029
24.9 25.799
26.7 29.123
22.9 25.143
24.0 24.653
21.1 23.294
22.7 23.751
24.4 22.61
29.8 28.634
27.6 30.91
25.4 24.702
23.6 24.088
23.8 24.686
28.8 28.733
22.9 23.126
23.1 24.484
34.3 28.905
24.4 24.056
22.1 23.981
27.3 24.401
34.1 31.044
23.3 23.206
23.6 23.298
24.5 24.889
25.2 24.534
24.5 24.119
25.5 25.709
28.1 26.591
26.1 25.137
22.9 24.925
26.6 25.761
26.8 27.592
17.6 20.406
21.9 22.887
16.2 20.669
24.5 24.253

In [68]:
# Save model
with open('model.pkl', 'wb') as f:
    pickle.dump(linear_reg,f)

In [ ]:
# Load model
with open('model.pkl', 'rb') as f:
    linear_reg = pickle.load(f)

### Добавить предсказание модели в датафрейм

In [78]:
# data == df (indexes)
data["y_pred"] = linear_reg.predict(data[X_cols]).round(3)
df["пресказанная_температура_помещения"] = data["y_pred"]

### Обзор полученного датафрейма (без графиков)

In [2]:
# Info
df.info()
df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 482 entries, 0 to 481
Data columns (total 34 columns):
 #   Column                                       Non-Null Count  Dtype  
---  ------                                       --------------  -----  
 0   год                                          482 non-null    int64  
 1   время_года                                   482 non-null    object 
 2   климат                                       482 non-null    object 
 3   город                                        482 non-null    object 
 4   страна                                       482 non-null    object 
 5   способ_охлаждения                            482 non-null    object 
 6   режим_при_смешанном_типе_охлаждения          335 non-null    object 
 7   способ_обогрева                              222 non-null    object 
 8   возраст                                      482 non-null    float64
 9   пол                                          410 non-null    object 
 10  ощ

,год,возраст,ощущение_температуры,ощущение_температуры_(bool),ощущение_движения_воздуха_(bool),оценка_комфорта,утепление,температура_воздуха_в_помещении,температура_воздуха_на_улице,rh,...,рост,вес,занавески,вентилятор,окно,двери,отопление,среднемесячная_температура_на_улице,количество_рекламаций,температура_удовлетворительная_%
count,482.000000,482.000000,482.000000,482.000000,482.000000,215.000000,482.000000,472.000000,482.000000,482.000000,...,482.000000,482.000000,482.000000,482.000000,482.000000,482.000000,482.000000,482.000000,482.000000,482.000000
mean,2011.029046,40.829876,0.208299,0.576763,0.207469,4.767442,0.634149,25.024153,24.100622,53.019295,...,165.558091,65.163900,0.004149,0.016598,-0.271784,-0.201245,0.093361,27.413693,0.105809,59.740041
std,0.922792,10.192946,1.091478,0.817760,0.607050,1.268549,0.253538,2.791078,0.232271,13.044019,...,5.253403,7.673977,0.386873,0.550689,0.696585,0.530718,0.394343,19.506410,0.441138,10.266735
min,2010.000000,22.000000,-3.000000,-1.000000,-1.000000,1.000000,0.360000,16.200000,22.800000,23.800000,...,134.600000,42.000000,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000,8.500000,0.000000,50.000000
25%,2010.000000,35.000000,0.000000,1.000000,0.000000,4.000000,0.460000,23.800000,24.100000,42.850000,...,165.100000,65.000000,0.000000,0.000000,-1.000000,-1.000000,0.000000,22.000000,0.000000,50.700000
50%,2011.000000,42.000000,0.000000,1.000000,0.000000,5.000000,0.590000,24.600000,24.100000,53.850000,...,165.100000,65.000000,0.000000,0.000000,0.000000,0.000000,0.000000,22.000000,0.000000,57.200000
75%,2012.000000,45.000000,1.000000,1.000000,1.000000,6.000000,0.700000,25.700000,24.100000,62.375000,...,165.100000,65.000000,0.000000,0.000000,0.000000,0.000000,0.000000,30.800000,0.000000,57.200000
max,2012.000000,65.000000,3.000000,1.000000,1.000000,6.000000,2.080000,37.400000,25.900000,85.100000,...,190.500000,110.000000,1.000000,1.000000,1.000000,1.000000,1.000000,328.000000,3.000000,78.000000


In [6]:
df.where(df["среднемесячная_температура_на_улице"] > 50).dropna(how='all')

,год,время_года,климат,город,страна,способ_охлаждения,режим_при_смешанном_типе_охлаждения,способ_обогрева,возраст,пол,...,вентилятор,окно,двери,отопление,среднемесячная_температура_на_улице,количество_рекламаций,температура_удовлетворительная_%,количество_рекламаций_кат,возраст_кат,rh_кат
52,2010.0,Лето,Cубтропический океанический,Сидней,Австралия,Смешанный,Кондиционирование,Механическое отопление,55.0,Женский,...,0.0,1.0,0.0,0.0,220.0,0.0,56.2,мало,средний возраст,норма
411,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,NaN,NaN,31.0,NaN,...,1.0,0.0,0.0,0.0,84.0,1.0,50.0,мало,молодой возраст,норма
423,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,NaN,NaN,29.0,NaN,...,1.0,0.0,0.0,0.0,89.0,1.0,50.0,мало,молодой возраст,менее_40
425,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,NaN,NaN,31.0,NaN,...,-1.0,0.0,0.0,0.0,89.0,1.0,50.0,мало,молодой возраст,менее_40
435,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,NaN,NaN,51.0,NaN,...,1.0,0.0,0.0,0.0,328.0,0.0,50.0,мало,средний возраст,норма
438,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,NaN,NaN,41.0,NaN,...,1.0,0.0,0.0,0.0,91.0,0.0,50.0,мало,молодой возраст,норма
440,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,NaN,NaN,30.0,NaN,...,1.0,0.0,0.0,0.0,91.0,3.0,50.0,много,молодой возраст,норма
441,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,NaN,NaN,28.0,NaN,...,-1.0,0.0,0.0,0.0,91.0,0.0,50.0,мало,молодой возраст,менее_40
451,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,NaN,NaN,43.0,NaN,...,1.0,0.0,0.0,0.0,84.0,0.0,50.0,мало,молодой возраст,менее_40
452,2011.0,Лето,Cубтропический океанический,Техас,США,Кондиционирование,NaN,NaN,38.0,NaN,...,-1.0,0.0,0.0,0.0,91.0,0.0,50.0,мало,молодой возраст,норма


In [3]:
# Value counts
for col in df.columns:
    print(col, df[col].value_counts(dropna=False))

год год
2012    212
2010    198
2011     72
Name: count, dtype: int64
время_года время_года
Лето     372
Зима      79
Весна     18
Осень     13
Name: count, dtype: int64
климат климат
Cубтропический океанический         270
Тропическая влажная саванна          77
Жаркий полузасушливый                68
Влажный субтропический муссонный     35
Субтропическое высокогорье           32
Name: count, dtype: int64
город город
Сидней        198
Техас          72
Ченнай         51
Ахмедабад      38
Дели           35
Шимла          32
Хайдарабад     30
Бангалор       26
Name: count, dtype: int64
страна страна
Индия        212
Австралия    198
США           72
Name: count, dtype: int64
способ_охлаждения способ_охлаждения
Смешанный            335
Кондиционирование    114
Вентиляция            33
Name: count, dtype: int64
режим_при_смешанном_типе_охлаждения режим_при_смешанном_типе_охлаждения
Кондиционирование    235
NaN                  147
Вентиляция           100
Name: count, dtype: int64
способ_

In [ ]:
# Show sorted numeric
for col in df.select_dtypes(include='number').columns:
    vals = df[col]
    print(col, (vals.min(), vals.quantile(0.05), [vals.median(), vals.mean()],  vals.quantile(0.95), vals.max()),
          df[col].sort_values().tolist(), sep='\n', end='\n\n')

## Сохранение итогового датасета
Завершение 1 этапа и перехо ко 2 этапу работы

In [81]:
# list(df.columns)  # Custom sort columns
df = df[[
    'год',
    'время_года',
    'климат',
    'страна',
    'город',

    'способ_охлаждения',
    'режим_при_смешанном_типе_охлаждения',
    'способ_обогрева',

    'оценка_комфорта',
    'пол',
    'рост',
    'вес',
    'возраст',
    'возраст_кат',
    'утепление',

    'ощущение_температуры',
    'ощущение_температуры_(bool)',
    'предпочтительное_изменение_температуры',
    'ощущение_движения_воздуха_(bool)',
    'предпочтительное_изменение_движения_воздуха',

    'пресказанная_температура_помещения',
    'температура_воздуха_в_помещении',
    'температура_воздуха_на_улице',
    'среднемесячная_температура_на_улице',
    'скорость_воздуха',
    'rh',
    'rh_кат',

    'занавески',
    'вентилятор',
    'окно',
    'двери',
    'отопление',

    'количество_рекламаций',
    'количество_рекламаций_кат',
    'температура_удовлетворительная_%',
]]

df.head()

,год,время_года,климат,страна,город,способ_охлаждения,режим_при_смешанном_типе_охлаждения,способ_обогрева,оценка_комфорта,пол,...,rh,rh_кат,занавески,вентилятор,окно,двери,отопление,количество_рекламаций,количество_рекламаций_кат,температура_удовлетворительная_%
0,2010,Лето,Cубтропический океанический,Австралия,Сидней,Вентиляция,Вентиляция,Механическое отопление,NaN,Женский,...,55.0,норма,0,0,-1,-1,0,0,мало,56.2
1,2010,Лето,Cубтропический океанический,Австралия,Сидней,Вентиляция,Вентиляция,Механическое отопление,NaN,Женский,...,58.5,норма,0,0,0,0,0,0,мало,56.2
2,2010,Лето,Cубтропический океанический,Австралия,Сидней,Вентиляция,Вентиляция,Механическое отопление,NaN,Женский,...,58.3,норма,0,0,-1,-1,0,0,мало,56.2
3,2010,Лето,Cубтропический океанический,Австралия,Сидней,Вентиляция,Вентиляция,Механическое отопление,NaN,Женский,...,59.5,норма,0,0,0,0,0,0,мало,56.2
4,2010,Лето,Cубтропический океанический,Австралия,Сидней,Вентиляция,Вентиляция,Механическое отопление,NaN,Женский,...,68.4,более_60,0,0,-1,-1,0,0,мало,56.2


In [82]:
df.sort_values(["страна", "город", "год", "время_года", "способ_охлаждения", "режим_при_смешанном_типе_охлаждения", "пол", "оценка_комфорта"], inplace=True)
df.to_csv('data_4.csv', index=False, sep=';', encoding='utf-8')

## Заметки

1) Готово предсказание температуры помещения
2) Можно попробовать ещё сделать оценку комфорта
3) ...
